<a href="https://colab.research.google.com/github/JaneChien-42/DSP-Estimation-of-Pollutants/blob/master/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ORA Final

# import packages

In [ ]:
!pip install pulp

     |████████████████████████████████| 40.6MB 69kB/s 


In [ ]:
import pandas as pd
import numpy as np
from pulp import *

# Data

## Read Data from Excel File

In [ ]:
!gdown --id '154saZVv_PLhgnSuk616EDuE582sCagmW' --output data.xlsx  # RawData.xlsx(with Price)

Downloading...
From: https://drive.google.com/uc?id=154saZVv_PLhgnSuk616EDuE582sCagmW
To: /content/data.xlsx
100% 107k/107k [00:00<00:00, 38.2MB/s]


In [ ]:
writer = pd.ExcelWriter('result.xlsx')
result_DSP = pd.DataFrame()

## Raw Data

In [ ]:
year = '2014'
data = pd.read_excel('data.xlsx', sheet_name=year) # read .xlsx data

X = data['Coal'].to_numpy().T
Y = data['Electricity'].to_numpy().T
B = data[['CO2 Emission','SO2 Emission','NOX Emission']].to_numpy().T
Price = data['Price'].to_numpy().T

print("X:",X,"\nY:",Y,"\nB:",B)
print("Price:",Price)


X: [   486750  23913177  19295415  22911006    290444  17769138    499319
    397113  22835149  22759871    743893  20548980     18445  52197770
  48560035  18198644  39213757  12632193   1248316   7450504     18887
  29529977  16799805  42966689   6550473  10186850  19495782  22342245
  15427283    543854   1060529  11913479   3445862   2220949  38485647
  18933951   1853491  38126571  11751519   1780371  17746768 101657503
  15312350   9281350   4486482  21334043  31918484  26492657] 
Y: [5.58292180e+05 4.73016262e+07 3.32207548e+07 4.26650106e+07
 8.04759570e+05 3.25448489e+07 8.24947890e+05 8.65384240e+05
 5.20536891e+07 4.52951133e+07 1.51118382e+06 3.37327654e+07
 7.76712700e+04 8.72823904e+07 9.75487385e+07 2.87522822e+07
 8.36019612e+07 1.92210190e+07 2.79488900e+06 1.76032909e+07
 7.91039200e+04 5.28998442e+07 2.79566792e+07 7.24092121e+07
 1.07425658e+07 1.55794154e+07 4.92381969e+07 2.73940682e+07
 2.49221749e+07 1.31099873e+06 2.51910621e+06 2.03556313e+07
 6.54786359e+06 4

# Define

In [ ]:
num = 48    # number of states
num_X = 1   # number of input items
num_Y = 1   # number of good output items
num_B = 3   # number of bad output items

Eff = np.zeros(num)

alpha_g = np.zeros(num)
alpha_b = np.zeros(num)

I = []  # Input
J = []  # Good Output
Q = []  # Bad Output
K = []  # Stats

for i in range(num_X):
  I.append(i)
for j in range(num_Y):
  J.append(j)
for q in range(num_B):
  Q.append(q)
for k in range(num):
  K.append(k)

# DEA

In [ ]:
g_Y = 0.25
# g_B = [0.3,0.3,0.3]
g_B = [0.048 * 0.75, 0.508 * 0.75, 0.444 * 0.75]
def DEA(r):
  prob_DEA = LpProblem("DEA", LpMaximize)
  
  eta = LpVariable("eta", lowBound=None)
  lamda = LpVariable.dicts("lamda", K, lowBound=0)
  mu = LpVariable.dicts("mu", K, lowBound=0)

  # Objective Function
  prob_DEA += eta
  # I-Constraint
  prob_DEA += lpSum([(lamda[k]+mu[k]) * X[k] for k in range(len(K))]) <= X[r]
  # GO-Constraint
  prob_DEA += lpSum([lamda[k] * Y[k] for k in range(len(K))]) >= Y[r] + g_Y * eta
  # BO-Constraint
  for q in range(len(Q)):
    prob_DEA += lpSum([lamda[k] * B[q][k] for k in range(len(K))]) <= B[q][r] - g_B[q] * eta
  # Convex
  prob_DEA += lpSum([lamda[k]+mu[k] for k in range(len(K))]) == 1

  prob_DEA.solve()

  # print("Status:", LpStatus[prob_DEA.status])

#   for v in prob_DEA.variables():
#       print(v.name, "=", v.varValue)
  print(r+1, 'Eff=','%.10f'%value(prob_DEA.objective))
  Eff[r]=value(prob_DEA.objective)
DEA(0)

1 Eff= 5708.5512000000


In [ ]:
for r in range(num):
  DEA(r)

1 Eff= 5708.5512000000
2 Eff= 44375.0910000000
3 Eff= 55607.8520000000
4 Eff= 0.0000000000
5 Eff= 0.0000000000
6 Eff= 20887.6570000000
7 Eff= 502.7546400000
8 Eff= 0.0000000000
9 Eff= 12401.4610000000
10 Eff= 23668.5330000000
11 Eff= 0.0000000000
12 Eff= 53426.6130000000
13 Eff= 0.0000000000
14 Eff= 0.0000000000
15 Eff= 58317.3960000000
16 Eff= 14008.0710000000
17 Eff= 33959.8810000000
18 Eff= 15029.4010000000
19 Eff= 1467.0030000000
20 Eff= 6894.5018000000
21 Eff= 0.0000000000
22 Eff= 64220.8310000000
23 Eff= 24232.9040000000
24 Eff= 71057.1210000000
25 Eff= 24324.1480000000
26 Eff= 10946.8670000000
27 Eff= 0.0000000000
28 Eff= 73080.4300000000
29 Eff= 34824.6650000000
30 Eff= 2412.2786000000
31 Eff= 827.7285300000
32 Eff= 3148.3000000000
33 Eff= 10653.3350000000
34 Eff= 14830.6270000000
35 Eff= 0.0000000000
36 Eff= 47049.1590000000
37 Eff= 4652.0823000000
38 Eff= 152198.5100000000
39 Eff= 0.0000000000
40 Eff= 25149.6130000000
41 Eff= 5694.3264000000
42 Eff= 0.0000000000
43 Eff= 0.000

In [ ]:
print(Eff)

[  5708.5512   44375.091    55607.852        0.           0.
  20887.657      502.75464      0.       12401.461    23668.533
      0.       53426.613        0.           0.       58317.396
  14008.071    33959.881    15029.401     1467.003     6894.5018
      0.       64220.831    24232.904    71057.121    24324.148
  10946.867        0.       73080.43     34824.665     2412.2786
    827.72853   3148.3      10653.335    14830.627        0.
  47049.159     4652.0823  152198.51         0.       25149.613
   5694.3264       0.           0.       29418.841        0.
  19528.338        0.       25025.556  ]


# Frontier Data

In [ ]:
Y_F = np.zeros(num)
B_F = np.zeros((3, num))
# print(B_F)
for i in range(num):
    Y_F[i] = Y[i] + g_Y * Eff[i]
    # Y_F[i] = Y_N[i] + g_Y * Eff[i]
for i in range(num_B):
    for j in range(num):
        B_F[i][j] = B[i][j] - g_B[i] * Eff[j]
        # B_F[i][j] = B_N[i][j] - g_B[i] * Eff[j]
print("Y_F:",Y_F)
print("B_F:",B_F)

Y_F: [5.59719318e+05 4.73127200e+07 3.32346568e+07 4.26650106e+07
 8.04759570e+05 3.25500708e+07 8.25073579e+05 8.65384240e+05
 5.20567895e+07 4.53010305e+07 1.51118382e+06 3.37461220e+07
 7.76712700e+04 8.72823904e+07 9.75633179e+07 2.87557842e+07
 8.36104511e+07 1.92247764e+07 2.79525575e+06 1.76050145e+07
 7.91039200e+04 5.29158994e+07 2.79627374e+07 7.24269764e+07
 1.07486469e+07 1.55821522e+07 4.92381969e+07 2.74123383e+07
 2.49308811e+07 1.31160180e+06 2.51931314e+06 2.03564184e+07
 6.55052692e+06 4.59576177e+06 8.98790516e+07 2.99177141e+07
 3.19375605e+06 7.90236782e+07 2.89143068e+07 2.69550314e+06
 3.58760056e+07 1.48173726e+08 3.33766877e+07 2.08259390e+07
 6.71992825e+06 3.74541464e+07 7.75146451e+07 4.34149074e+07]
B_F: [[1.29803749e+06 4.69886715e+07 3.26348101e+07 4.23247350e+07
  1.93099500e+06 3.30265180e+07 8.84210901e+05 9.55168000e+05
  5.13070835e+07 4.52500009e+07 1.60861100e+06 3.81487306e+07
  4.20544000e+05 9.32819150e+07 9.75571856e+07 3.04573027e+07
  8.27281

# Max Value of Each Column

In [ ]:
max_X = X.max()
max_Y = Y.max()
max_B = [B[0].max(),B[1].max(),B[2].max()]

max_Y_F = Y_F.max()
max_B_F = [B_F[0].max(),B_F[1].max(),B_F[2].max()]

print("max_X:", max_X)
print("max_Y:", max_Y)
print("max_B:", max_B)
print("max_Y_F:", max_Y_F)
print("max_B_F:", max_B_F)

max_X: 101657503
max_Y: 148173726.4
max_B: [151751904, 303785, 108237]
max_Y_F: 148173726.4
max_B_F: [151751904.0, 303785.0, 87278.0]


# DMP -- Good Output

## v(alpha)

In [ ]:
g_Y_g = 1
g_B_g = [0,0,0]
def GMP(r):
    prob_GMP = LpProblem("GMP", LpMinimize)

    u_0 = LpVariable("u_0", lowBound=None)
    v = LpVariable("v", lowBound=0)
    u = LpVariable("u", lowBound=0)
    w = LpVariable.dicts("w", Q, lowBound=None)

    # Objective Function
    prob_GMP += v
    # Constraints
    prob_GMP += v * X[r]/max_X - u * Y_F[r]/max_Y_F + lpSum([w[q] * B_F[q][r]/max_B_F[q] for q in range(len(Q))]) + u_0 == 0
    for k in range(len(K)):
        prob_GMP += v * X[k]/max_X - u * Y[k]/max_Y + lpSum([w[q] * B[q][k]/max_B[q] for q in range(len(Q))]) + u_0 >= 0
        prob_GMP += v * X[k]/max_X + u_0 >= 0
    prob_GMP += u * g_Y_g + lpSum([w[q] * g_B_g[q] for q in range(len(Q))]) == 1
    prob_GMP.solve()

    # 查看目前解的狀態
    print("Status:", LpStatus[prob_GMP.status])

    # 印出解及目標值
    # for v in prob_GMP.variables():
    #     print(v.name, "=", v.varValue)
    print(r+1,' %.10f'%value(prob_GMP.objective))
    alpha_g[r] = value(prob_GMP.objective)
GMP(0)

Status: Optimal
1  0.0000000000


In [ ]:
for r in range(num):
    GMP(r)
print(alpha_g)

Status: Optimal
1  0.0000000000
Status: Infeasible
2  0.0000000000
Status: Infeasible
3  0.0000000000
Status: Optimal
4  0.0000000000
Status: Optimal
5  1.2973363000
Status: Infeasible
6  0.0000000000
Status: Infeasible
7  0.0000000000
Status: Infeasible
8  2.2302996000
Status: Infeasible
9  0.0000000000
Status: Infeasible
10  0.0000000000
Status: Optimal
11  0.0954111610
Status: Optimal
12  0.0000000000
Status: Optimal
13  0.8492506100
Status: Infeasible
14  0.6005407600
Status: Optimal
15  0.0000000000
Status: Infeasible
16  0.0000000000
Status: Infeasible
17  0.9800353700
Status: Optimal
18  0.0000000000
Status: Infeasible
19  82.5462220000
Status: Optimal
20  4.6937316000
Status: Optimal
21  0.8943105000
Status: Infeasible
22  0.2107731500
Status: Infeasible
23  0.0000000000
Status: Infeasible
24  0.0000000000
Status: Optimal
25  0.0000000000
Status: Infeasible
26  0.0000000000
Status: Optimal
27  0.0000000000
Status: Optimal
28  0.0000000000
Status: Infeasible
29  0.0000000000
Sta

In [ ]:
print(alpha_g)

[ 0.          0.          0.          0.          1.2973363   0.
  0.          2.2302996   0.          0.          0.09541116  0.
  0.84925061  0.60054076  0.          0.          0.98003537  0.
 82.546222    4.6937316   0.8943105   0.21077315  0.          0.
  0.          0.          0.          0.          0.          1.1568149
  1.1238347   0.          0.          0.          0.          0.
  0.          0.         51.765124    0.          1.7697455   0.
  0.          1.3021575   0.          0.          0.          0.        ]


## GMP(each state)

In [ ]:
MP_g = np.zeros(num)
for r in range(num):
    MP_g[r] = g_Y_g * max_Y * alpha_g[r] / max_X
print(MP_g)

[  0.           0.           0.           0.           1.89096868
   0.           0.           3.25083533   0.           0.
   0.1390692    0.           1.23784889   0.87533492   0.
   0.           1.42847786   0.         120.3175462    6.84147929
   1.30352719   0.30721828   0.           0.           0.
   0.           0.           0.           0.           1.68614779
   1.63807659   0.           0.           0.           0.
   0.           0.           0.          75.45169903   0.
   2.57954187   0.           0.           1.89799595   0.
   0.           0.           0.        ]


# DMP -- Bad Output

## v(alpha)

In [ ]:
g_Y_b = 0
g_B_b = [0.048, 0.508, 0.444]
# g_B_b = [0.045, 0.631, 0.324]
def BMP(r):
    prob_BMP = LpProblem("BMP", LpMinimize)

    u_0_b = LpVariable("u_0_b", lowBound=None)
    v_b = LpVariable("v_b", lowBound=0)
    u_b = LpVariable("u_b", lowBound=0)
    w_b = LpVariable.dicts("w_b", Q, lowBound=None)

    # Objective Function
    prob_BMP += v_b
    # Constraints
    prob_BMP += v_b * X[r]/max_X - u_b * Y_F[r]/max_Y_F + lpSum([w_b[q] * B_F[q][r]/max_B_F[q] for q in range(len(Q))]) + u_0_b == 0
    for k in range(len(K)):
        prob_BMP += v_b * X[k]/max_X - u_b * Y[k]/max_Y + lpSum([w_b[q] * B[q][k]/max_B[q] for q in range(len(Q))]) + u_0_b >= 0
        prob_BMP += v_b * X[k]/max_X + u_0_b >= 0
    prob_BMP += u_b * g_Y_b + lpSum([w_b[q] * g_B_b[q] for q in range(len(Q))]) == 1
    prob_BMP.solve()

    # 查看目前解的狀態
    print("Status:", LpStatus[prob_BMP.status])

    # 印出解及目標值
    # for v in prob_BMP.variables():
    #     print(v.name, "=", v.varValue)
    print(r+1,' %.10f'%value(prob_BMP.objective))
    alpha_b[r] = value(prob_BMP.objective)
BMP(0)

Status: Optimal
1  0.0000000000


In [ ]:
for r in range(num):
    BMP(r)
print(alpha_b)

Status: Optimal
1  0.0000000000
Status: Infeasible
2  0.0000000000
Status: Infeasible
3  0.0000000000
Status: Optimal
4  0.0000000000
Status: Optimal
5  0.3035467200
Status: Infeasible
6  7.0430735000
Status: Infeasible
7  0.0000000000
Status: Infeasible
8  4.8738452000
Status: Infeasible
9  15.5877460000
Status: Infeasible
10  0.0000000000
Status: Optimal
11  0.0591094370
Status: Optimal
12  0.0000000000
Status: Optimal
13  1.4518486000
Status: Infeasible
14  2.8010518000
Status: Optimal
15  0.0000000000
Status: Infeasible
16  0.0000000000
Status: Infeasible
17  43.9241860000
Status: Optimal
18  0.0000000000
Status: Infeasible
19  3.4822823000
Status: Optimal
20  9.1559128000
Status: Optimal
21  4.6803749000
Status: Infeasible
22  2.7423490000
Status: Infeasible
23  0.0000000000
Status: Infeasible
24  15.8025240000
Status: Optimal
25  0.0000000000
Status: Infeasible
26  0.0000000000
Status: Optimal
27  0.0000000000
Status: Optimal
28  0.0000000000
Status: Infeasible
29  0.0000000000
S

## BMP(each state)

In [ ]:
MP_b = np.zeros((3, num))

for q in range(num_B):
    for r in range(num):
        MP_b[q][r] = g_B_b[q] * max_B[q] * alpha_b[r] / max_X
print(MP_b)

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  2.17501117e-02 5.04659170e-01 0.00000000e+00 3.49226892e-01
  1.11691280e+00 0.00000000e+00 4.23538380e-03 0.00000000e+00
  1.04029684e-01 2.00704490e-01 0.00000000e+00 0.00000000e+00
  3.14731108e+00 0.00000000e+00 2.49516876e-01 6.56050991e-01
  3.35364115e-01 1.96498243e-01 0.00000000e+00 1.13230235e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.83346116e-01 1.01194092e-01 0.00000000e+00
  3.18365741e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 5.57973537e-01 0.00000000e+00
  2.48242859e-01 0.00000000e+00 0.00000000e+00 2.47452581e-01
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  4.60803898e-04 1.06918491e-02 0.00000000e+00 7.39881778e-03
  2.36632243e-02 0.00000000e+00 8.97320156e-05 0.00000000e+00
  2.20400173e-03 4.25218098e-03 0.00000000e+00 0.00000000e+00
  6.667

# DSP

## Each State

In [ ]:
SP = np.zeros((3,num))

for q in range(num_B):
    for r in range(num):
        if MP_g[r]==0:
            SP[q][r] = 0
        else:
            SP[q][r] = Price[r] * MP_g[r] / MP_b[q][r]
print(SP)

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.31710873e+04 0.00000000e+00 0.00000000e+00 1.04449562e+03
  0.00000000e+00 0.00000000e+00 1.09762548e+04 0.00000000e+00
  9.43113632e+02 4.08156022e+02 0.00000000e+00 0.00000000e+00
  3.69913074e+01 0.00000000e+00 7.40399918e+04 1.26197128e+03
  4.91704394e+02 1.72488033e+02 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.39974494e+03 2.25790491e+03 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 1.30715598e+04 0.00000000e+00
  9.76460883e+02 0.00000000e+00 0.00000000e+00 7.03685432e+02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  6.21680116e+05 0.00000000e+00 0.00000000e+00 4.93005736e+04
  0.00000000e+00 0.00000000e+00 5.18083223e+05 0.00000000e+00
  4.45153067e+04 1.92651128e+04 0.00000000e+00 0.00000000e+00
  1.746

## Average

In [ ]:
metric_ton_to_ton = 1.10231131092

DSP = np.zeros(num_B)
DSP_numerator = np.zeros(num_B)
DSP_denominator = np.zeros(num_B)

for q in range(num_B):
    for r in range(num):
        if MP_b[q][r]!=0:
            DSP_numerator[q] = DSP_numerator[q] + SP[q][r] * Y[r]
            DSP_denominator[q] = DSP_denominator[q] + Y[r]
    DSP[q] = DSP_numerator[q]/DSP_denominator[q]/metric_ton_to_ton
print(DSP)

[  1341.89172936  63337.77809639 203392.13062512]


# Export

In [ ]:
summary = np.concatenate((X.reshape((1,num)), Y.reshape((1,num)), B, Price.reshape((1,num)),\
                Eff.reshape((1,num)), Y_F.reshape((1,num)), B_F,\
                alpha_g.reshape((1,num)), MP_g.reshape((1,num)),\
                alpha_b.reshape((1,num)), MP_b,\
                SP))
result = pd.DataFrame(summary,index=['Coal','Electricity','CO2','SO2','NOx','Price',\
                    'Eff','Electricity_F','CO2_F','SO2_F','NOx_F',\
                    'v(Good)','GMP',\
                    'v(Bad)','CO2_MP)','SO2_MP','NOx_MP',\
                    'CO2_SP','SO2_SP','NOx_SP']).T
print(result)
result.to_excel(writer, sheet_name=year)

           Coal   Electricity  ...        SO2_SP        NOx_SP
0      486750.0  5.582922e+05  ...  0.000000e+00  0.000000e+00
1    23913177.0  4.730163e+07  ...  0.000000e+00  0.000000e+00
2    19295415.0  3.322075e+07  ...  0.000000e+00  0.000000e+00
3    22911006.0  4.266501e+07  ...  0.000000e+00  0.000000e+00
4      290444.0  8.047596e+05  ...  6.216801e+05  1.996357e+06
5    17769138.0  3.254485e+07  ...  0.000000e+00  0.000000e+00
6      499319.0  8.249479e+05  ...  0.000000e+00  0.000000e+00
7      397113.0  8.653842e+05  ...  4.930057e+04  1.583154e+05
8    22835149.0  5.205369e+07  ...  0.000000e+00  0.000000e+00
9    22759871.0  4.529511e+07  ...  0.000000e+00  0.000000e+00
10     743893.0  1.511184e+06  ...  5.180832e+05  1.663684e+06
11   20548980.0  3.373277e+07  ...  0.000000e+00  0.000000e+00
12      18445.0  7.767127e+04  ...  4.451531e+04  1.429489e+05
13   52197770.0  8.728239e+07  ...  1.926511e+04  6.186470e+04
14   48560035.0  9.754874e+07  ...  0.000000e+00  0.000

In [ ]:
DSP_to_DF = pd.DataFrame(DSP,index=['CO2','SO2','NOx'],columns=[year]).T
result_DSP = result_DSP.append(DSP_to_DF)
print(result_DSP)

               CO2           SO2           NOx
2000    587.216608  7.132774e+03  2.691324e+04
2001    596.683310  7.366975e+03  2.952502e+04
2002    454.483929  5.798168e+03  2.109810e+04
2003    625.793785  7.911528e+03  3.150223e+04
2004   1316.500783  1.839889e+04  8.905255e+04
2005    535.782338  7.449263e+03  3.819827e+04
2006    750.550421  1.150445e+04  5.623096e+04
2007   1125.618828  1.749776e+04  8.434777e+04
2008    515.700949  9.615737e+03  3.939943e+04
2009    296.839021  6.698689e+03  4.388137e+04
2010    365.076080  9.134046e+03  5.113425e+04
2011    718.875249  1.885752e+04  9.701965e+04
2012    134.814643  5.614700e+03  1.824465e+04
2013    469.944297  2.060888e+04  6.510810e+04
2015    293.337995  1.514072e+04  4.946187e+04
2016    800.778068  4.340779e+04  1.519720e+05
2017    927.643954  4.917733e+04  1.808180e+05
2018  25143.372062  1.422266e+06  4.540293e+06
2019    618.282932  4.150791e+04  1.155130e+05
2014   1341.891729  6.333778e+04  2.033921e+05


## Write to Excel

In [ ]:
result_DSP.to_excel(writer, sheet_name="DSP")
writer.save()